## 11.6 VeryNobleApp

W tej aplikacji wykorzystamy darmowe api ze strony [nabelprize.org](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1) do stworzenia aplikacji wyświetlającej nagrody nobla w architekturze `MVVM`. Aplikacja będzie zawierała cztery fragmenty:
- `NobelAwardsFragment` wyświetlający listę (`RecyclerView`) wszystkich nagród według kategorii
- `NobelAwardFragment` wyświetlający więcej informacji na temat wybranej na poprzednim ekranie nagrody, zawiera również listę (`RecyclerView`) wszystkich laureatów nagrody
- `LaureateFragment` wyświetlający więcej informacji na temat wybranego na poprzednim ekranie laureata, również zawiera `RecyclerView` ze wszystkimi nagrodami otrzymanymi przez danego laureata
- `WikiFragment` wyświetlający wpis na wikipedii dotyczący wybranego laureate - strona wyświetlana jest za pomocą `WebView`.

<table><tr><td><img src="https://media0.giphy.com/media/NuuG5HnAdmdTjLV0DV/giphy.webp" width="200" /></td><td><img src="https://media3.giphy.com/media/6pIR2WozCb81CcuGZo/giphy.webp" width="200" /></td><td><img src="https://media0.giphy.com/media/8QoE7dYN8as4gRjE16/giphy.webp" width="200" /></td></tr></table>


Tutaj mniejszy nacisk przyłożymy na warstwę danych (zostanie wygenerowana automatycznie) - skupimy się na implementacji architektury `MVVM` wykorzystując bibliotekę `Retrofit`. Wykorzystamy pojedyncze repozytorium oraz zaimplementujemy osobny `ViewModel` dla każdego fragmentu, który tego wymaga.

Rozpocznijmy od zmodyfikowania skryptów `gradle`, do pliku `gradle(Project)` dodajemy możliwość przesyłania argumentów przez `Navigation`

In [ ]:
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.2"
    }
}

Uzupełniamy również plik `gradle(Module)`

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
    id 'kotlin-android'
    id 'kotlin-kapt'
}
...
android {
    ...
    buildFeatures {
        viewBinding true
    }
}
...
dependencies {
    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel-ktx:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata-ktx:2.5.1"

    // Fragment
    implementation "androidx.fragment:fragment-ktx:1.5.2"

    // Navigation
    implementation "androidx.navigation:navigation-fragment-ktx:2.5.2"
    implementation "androidx.navigation:navigation-ui-ktx:2.5.2"

    // Retrofit
    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'

    // OkHttp
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'
    ...
}

### **`Navigation`**

Nawigacja w tej aplikacji będzie liniowa - z jednego fragmetu można przejść do jednego innego fragmentu i z powrotem. Dodajmy nawigację i utwórzmy odpowiednie akcje

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/nobelAwardsFragment">

    <fragment
        android:id="@+id/nobelAwardsFragment"
        android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.
fragments.nobleawards.NobelAwardsFragment"
        android:label="Nagrody Nobla"
        tools:layout="@layout/fragment_nobel_awards" >
        <action
            android:id="@+id/action_nobelAwardsFragment_to_nobelAwardFragment"
            app:destination="@id/nobelAwardFragment" >
        </action>
    </fragment>
    <fragment
        android:id="@+id/nobelAwardFragment"
        android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.fragments.nobelaward.NobelAwardFragment"
        android:label="Nagroda Nobla"
        tools:layout="@layout/fragment_nobel_award" >
        <action
            android:id="@+id/action_nobelAwardFragment_to_nobelAwardsFragment"
            app:destination="@id/nobelAwardsFragment" />
        <action
            android:id="@+id/action_nobelAwardFragment_to_laureateFragment"
            app:destination="@id/laureateFragment" >
        </action>
    </fragment>
    <fragment
        android:id="@+id/laureateFragment"
        android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.fragments.laureate.LaureateFragment"
        android:label="Laureat"
        tools:layout="@layout/fragment_laureate" >
        <action
            android:id="@+id/action_laureateFragment_to_nobelAwardFragment"
            app:destination="@id/nobelAwardFragment" />
        <action
            android:id="@+id/action_laureateFragment_to_wikiLaureateFragment"
            app:destination="@id/wikiLaureateFragment" >
        </action>
    </fragment>
    <fragment
        android:id="@+id/wikiLaureateFragment"
        android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.fragments.wiki.WikiLaureateFragment"
        android:label="Wikipedia"
        tools:layout="@layout/fragment_wiki_laureate" />
</navigation>

### **`Retrofit`**

Wszystkie fragmenty wraz z aktywnością umieszczam w pakiecie `ui`. Dodajmy pakiet `api` do którego dodamy interfejs z metodami dostępowymi (których ciało zostanie wygenerowane przez `Retrofit`), oraz instancję samoego `Retrofit`. Wpierw przyjrzyjmy się naszemu [api](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1) - do uzyskania listy wszystkich nagród nobla o zadanej kategorii wykorzystamy endpoint **/nobelPrizes**, bazowym adresem `url` jest https://api.nobelprize.org/ - pełne zapytanie będzie wyglądać następująco
  'https://api.nobelprize.org/2.1/nobelPrizes?limit=30&sort=desc&nobelPrizeYear=2000&yearTo=2021&nobelPrizeCategory=eco&format=json'

Mamy tutaj serię parametrów:
```xml
- name: `offset`  
  in: query  
  description: Liczba elementów pominiętych przed rozpoczęciem zbierania  
  type: `integer`  
  minimum: 1  

- name: `limit`  
  in: query  
  description: Liczba zwróconych elementów  
  type: `integer`  
  minimum: 1

- name: `sort`  
  in: query  
  description: Kolejność sortowania  
  type: `string`  
  enum: ["asc", "desc"]  

- name: `nobelPrizeYear`  
  in: query  
  description: Rok przyznania nagrody nobla  
  type: `integer`  
  minimum: 1901

- name: `yearTo`  
  in: query  
  description: Z `nobelPrizeYear` wykorzystywany do wygenerowania zakresu  
  type: `integer`  
  minimum: 1901

- name: `nobelPrizeCategory`  
  in: query  
  description: Kategoria nagrody nobla  
  type: `string`  
  enum: ["che", "eco", "lit", "pea", "phy", "med"]

- name: `format`  
  in: query  
  description: Format wyjścia - domyślnie json  
  type: `string`  
  enum: ["json", "csv"]

- name: `csvLang`  
  in: query  
  description: Język wyjścia - domyślnie angielski  
  type: `string`  
  enum: ["en", "se", "no"]
```

Przykładowy wynik zapytania   https://api.nobelprize.org/2.1/nobelPrizes?limit=1&sort=desc&nobelPrizeYear=2000&nobelPrizeCategory=phy

```json
{
  "nobelPrizes": [
    {
      "awardYear": "2000",
      "category": {
        "en": "Physics",
        "no": "Fysikk",
        "se": "Fysik"
      },
      "categoryFullName": {
        "en": "The Nobel Prize in Physics",
        "no": "Nobelprisen i fysikk",
        "se": "Nobelpriset i fysik"
      },
      "dateAwarded": "2000-10-10",
      "topMotivation": {
        "en": "for basic work on information and communication technology"
      },
      "prizeAmount": 9000000,
      "prizeAmountAdjusted": 11538617,
      "links": [
        {
          "rel": "nobelPrize",
          "href": "https://api.nobelprize.org/2/nobelPrize/phy/2000",
          "action": "GET",
          "types": "application/json"
        }
      ],
      "laureates": [
        {
          "id": "726",
          "knownName": {
            "en": "Zhores Alferov"
          },
          "fullName": {
            "en": "Zhores I. Alferov"
          },
          "portion": "1/4",
          "sortOrder": "1",
          "motivation": {
            "en": "for developing semiconductor heterostructures used in high-speed- and opto-electronics",
            "se": "för utvecklingen av halvledarheterostrukturer för höghastighets- och optoelektronik"
          },
          "links": [
            {
              "rel": "laureate",
              "href": "https://api.nobelprize.org/2/laureate/726",
              "action": "GET",
              "types": "application/json"
            }
          ]
        },
        {
          "id": "727",
          "knownName": {
            "en": "Herbert Kroemer"
          },
          "fullName": {
            "en": "Herbert Kroemer"
          },
          "portion": "1/4",
          "sortOrder": "2",
          "motivation": {
            "en": "for developing semiconductor heterostructures used in high-speed- and opto-electronics",
            "se": "för utvecklingen av halvledarheterostrukturer för höghastighets- och optoelektronik"
          },
          "links": [
            {
              "rel": "laureate",
              "href": "https://api.nobelprize.org/2/laureate/727",
              "action": "GET",
              "types": "application/json"
            }
          ]
        },
        {
          "id": "728",
          "knownName": {
            "en": "Jack Kilby"
          },
          "fullName": {
            "en": "Jack S. Kilby"
          },
          "portion": "1/2",
          "sortOrder": "3",
          "motivation": {
            "en": "for his part in the invention of the integrated circuit",
            "se": "för hans del i uppfinningen av den integrerade kretsen"
          },
          "links": [
            {
              "rel": "laureate",
              "href": "https://api.nobelprize.org/2/laureate/728",
              "action": "GET",
              "types": "application/json"
            }
          ]
        }
      ]
    }
  ],
  "meta": {
    "offset": 0,
    "limit": 1,
    "nobelPrizeYear": 2000,
    "nobelPrizeCategory": "phy",
    "count": 1,
    "terms": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/",
    "license": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#licence",
    "disclaimer": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#disclaimer"
  }
}
```

Możemy wynik zmapować przy pomocy pluginu w Android Studio - `JSONToKotlinClass` - w efekcie dostaniemy całą serię klas, które dodajemy do pakietu `data`

In [ ]:
data class NobelPrizeResponse(
    val meta: Meta,
    val nobelPrizes: List<NobelPrize>
)

data class Meta(
    val count: Int,
    val disclaimer: String,
    val license: String,
    val limit: Int,
    val nobelPrizeCategory: String,
    val nobelPrizeYear: Int,
    val offset: Int,
    val terms: String,
    val yearTo: Int
)

data class NobelPrize(
    val awardYear: String,
    val category: Category,
    val categoryFullName: CategoryFullName,
    val dateAwarded: String,
    val laureates: List<Laureate>?,
    val links: List<LinkX>,
    val prizeAmount: Int,
    val prizeAmountAdjusted: Int,
    val topMotivation: TopMotivation?
)

data class Category(
    val en: String,
    val no: String,
    val se: String
)

data class CategoryFullName(
    val en: String,
    val no: String,
    val se: String
)

data class TopMotivation(
    val en: String
)

data class Laureate(
    val fullName: FullName?,
    val id: String,
    val knownName: KnownName,
    val links: List<LinkX>,
    val motivation: Motivation,
    val portion: String,
    val sortOrder: String
)

data class FullName(
    val en: String?
)

data class KnownName(
    val en: String
)

data class Motivation(
    val en: String
)

data class LinkX(
    val action: String,
    val href: String,
    val rel: String,
    val types: String
)

Główną klasą jest `NobelPrizeResponse`, zawierającą metadane oraz listę nagród nobla - w tej aplikacji nie wykorzystamy wszystkich danych, lecz możemy pozostawić wygenerowane klasy bez zmian.

Do pakietu `api` dodajmy interfejs `NobelPrizeApi` i zdefiniujmy metodę zwracającą listę nagród dla zadanej kategorii. Będziemy zapytania wykonywać **asynchronicznie**, więc definiujemy `getNobelProzes` jako funkcję zawieszoną

In [ ]:
interface NobelPrizeApi {
    @GET("2.1/nobelPrizes")
    suspend fun getNobelPrizes(
        @Query("limit") limit: Int = 200,
        @Query("sort") sort: String = "desc",
        @Query("nobelPrizeYear") yearFrom: Int = 1901,
        @Query("yearTo") yearTo: Int = 2022,
        @Query("nobelPrizeCategory") category: String,
        @Query("format") format: String = "json"
    ) : Response<NobelPrizeResponse>
}

Następnie utwórzmy instancję `Retrofit` - do pakietu `api` dodajmy obiekt `RetrofitInstance`. Wykorzystamy również `HttpLoggingInterceptor` w celu sprawdzenia odpowiedzi.

In [ ]:
object RetrofitInstance {
    val api: NobelPrizeApi by lazy {
        val logging = HttpLoggingInterceptor()
        logging.setLevel(HttpLoggingInterceptor.Level.BODY)
        val client = OkHttpClient.Builder()
            .addInterceptor(logging)
            .build()
        Retrofit.Builder()
            .baseUrl(baseUrl)
            .addConverterFactory(GsonConverterFactory.create())
            .client(client)
            .build()
            .create(NobelPrizeApi::class.java)
    }
}

Do pakietu `util` dodajmy plik `Constants` w którym zdefiniujemy `baseUrl`

In [ ]:
const val baseUrl: String = "https://api.nobelprize.org/"

Do manifestu dodajmy upoważnienie na dostęp do internetu

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

### **szkielet `MVVM`**

Rozpocznijmy od repozytorium (w pakiecie `repository`) - zawiera jedną metodę `getNobelPrizes`

In [ ]:
class NobelRepository {
    suspend fun getNobelPrizes(category: String) = 
    RetrofitInstance.api.getNobelPrizes(category=category)
}

Dodajmy do pakietu zawierającego `NobelAwardsFragment` (`ui.fragments.nobelawards`) odpowiedni `ViewModel`

In [ ]:
class NobelPrizesViewModel : ViewModel() {
    private val repository = NobelRepository()
    private val _nobelPrizes: 
    MutableLiveData<NobelPrizeResponse> = MutableLiveData()

    val noblePrizes: LiveData<NobelPrizeResponse>
        get() = _nobelPrizes
}

Odbieramy dane jako `NobelPrizeResponse` - przechowujemy je jako `MutableLiveData` w prywatnej wartości `_nobelPrizes`. Dodajemy pole wspomagające z getterem, które wystawimy jako publiczne - zwraca `LiveData` (niemutowalne).

Jest to niewystarczające do pełnego odebrania odpowiedzi - wprowadźmy wrapper zalecany przez **Google** - do pakietu `util` dodajmy klasę `Resource` - będzie to klasa generyczna, zapieczętowana. Klasa ta pozwoli nam na rozróżnienie odpowiedzi poprawnej od błędnej, oraz określenie stanu ładowania. W konstruktorze umieszczamy dwa pola
- `data` - typu `T` zawierającą otrzymane dane
- `message` - typu `String` zawierającą komunikat błędu
Domyślnie inicjujemy obie wartości jako `null`

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){}

Ponieważ jest to klasa zapieczętowana, mamy pełną kontrolę nad klasami które mogą ją rozszerzać. W ciele `Resource` zdefiniujemy trzy klasy reprezentujące trzy stany
- `Success` - klasa rozszerzająca klasę `Resource`, jeżeli dostaniemy poprawną odpowiedź nasz `Response` posiada `body`, więc jako argument konstruktora podajemy `data` który otrzymujemy - `message` pozostaje jako `null` (nie ma błędu)

In [ ]:
class Success<T>(data: T) : Resource<T>(data)

- `Error` - klasa rozszerzająca klasę `Resource`, jeżeli zostanie wygenerowany błąd, dostaniemy odpowiedni komunikat, który przekazujemy w konstruktorze. Przy tej odpowiedzi nasz `Response` może posiadać `body` (opcjonalnie), więc jako drugi argument przekazujemy `data` jako typ zerowalny zainicjowany domyślnie `null`

In [ ]:
class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)

- `Loading` - klasa rozszerzająca klasę `Resource`, reprezentująca stan ładowania - nie posiada parametrów konstruktora

In [ ]:
class Loading<T> : Resource<T>()

Pełny kod klasy `Resource`

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)
    class Loading<T> : Resource<T>()
}

Gdy wykonamy żądanie, emitujemy stan `Loading`, po otrzymaniu odpowiedzi emitujemy `Success` lub `Error`

Powróćmy do klasy `NobelPrizesViewModel` - teraz nasze `LiveData` będzie typu `Resource<NobelPrizeResponse>`

In [ ]:
class NobelPrizesViewModel : ViewModel() {
    private val repository = NobelRepository()
    private val _nobelPrizes: 
    MutableLiveData<Resource<NobelPrizeResponse>> = MutableLiveData()

    val noblePrizes: LiveData<Resource<NobelPrizeResponse>>
        get() = _nobelPrizes

Dodajmy metodę obsługującą `Response`, która wyemituje odpowiedni stan w zależności od stanu odpowiedzi.

In [ ]:
private fun handleNobelPrizesResponse(response: Response<NobelPrizeResponse>)
: Resource<NobelPrizeResponse>{
    if (response.isSuccessful)
        response.body()?.let { return Resource.Success(it) }
    return Resource.Error(response.message())
}

Funkcja zwraca odpowiedni `Resource`, jeżeli `response.isSuccessful` jest spełniony chcemy wyemitować stan `Resource.Success`, przyjmujący dane jako parametr konstruktora. W przeciwnym razie zwracamy obiekt `Resource.Error` z odpowiednim komunikatem.

Następnie zdefiniujmy funkcję zwracającą listę wszystkich nagród dla zadanej kategorii.

In [ ]:
fun getNobelPrizes(category: String) = viewModelScope.launch {}

W pierwszym kroku, po wykonaniu żądania, emitujemy stan ładowania

In [ ]:
_nobelPrizes.postValue(Resource.Loading())

Następnie wywołujemy metodę `getNobelPrizes` z naszego repozytorium

In [ ]:
val response = repository.getNobelPrizes(category= category)

Następnie do `MutableLiveData` emitujemy stan zależny od otrzymanej odpowiedzi - w tym celu korzystamy z wcześniej zdefiniowanej funkcji `handleNobelPrizesResponse`

In [ ]:
_nobelPrizes.postValue(handleNobelPrizesResponse(response))

Pełny kod metody `getNobelPrizes`

In [ ]:
fun getNobelPrizes(category: String) = viewModelScope.launch {
    _nobelPrizes.postValue(Resource.Loading())
    val response = repository.getNobelPrizes(category= category)
    _nobelPrizes.postValue(handleNobelPrizesResponse(response))
}

Ponieważ nie przy starcie aplikacji chcemy wyświetlić dane - tutaj będą to wszystkie nagrody dla fizyki - musimy określić kategorie. Przejdźmy do pliku `Constants` w pakiecie `util` i dodajmy klasę wyliczeniową wraz z mapą pozwalającą na wybór kategorii.

In [ ]:
enum class Cat{
    PHYSICS, CHEMISTRY, LITERATURE, PEACE, ECONOMY, PHYSIOLOGYORMEDICINE
}

val categories = mapOf(
    Cat.PHYSICS to "phy",
    Cat.CHEMISTRY to "che",
    Cat.ECONOMY to "eco",
    Cat.LITERATURE to "lit",
    Cat.PEACE to "pea",
    Cat.PHYSIOLOGYORMEDICINE to "med"
)

W naszym `NobelAwardsViewModel` wywołajmy funkcję `getNobelPrizes` w bloku `init` - co pozwoli nam zainicjować listę przy starcie aplikacji.

In [ ]:
init {
    categories[Cat.PHYSICS]?.let { getNobelPrizes(category = it) }
}

### **`NobelAwardsFragment`**

Przejdźmy do fragmentu i rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <Spinner
        android:id="@+id/categorySpinner"
        style="@style/Widget.AppCompat.Spinner"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:drawSelectorOnTop="true"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/nobelPrizeRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/categorySpinner" />

    <ProgressBar
        android:id="@+id/nobelPrizeProgressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="8dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/categorySpinner" />

</androidx.constraintlayout.widget.ConstraintLayout>

Wykorzystamy trzy elementy
- `Spinner` do wyboru kategorii
- `RecyclerView` do wyświetlenia listy
- `ProgressBar` wyświetlany przy ładowaniu danych

Zdefiniujmy layout dla elementu spinnera

In [ ]:
<TextView xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/spinner_layout"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:ellipsize="marquee"
    android:fontFamily="sans-serif"
    android:gravity="center"
    android:singleLine="true"
    android:text=""
    android:padding="10dp"
    android:textSize="24sp" />

oraz dla pojedynczego elementu `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>

<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp"
    app:cardBackgroundColor="@color/teal_200"
    android:layout_margin="8dp"
    >
<androidx.constraintlayout.widget.ConstraintLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"

    android:layout_marginTop="16dp"
    >

        <TextView
            android:id="@+id/year"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:layout_marginBottom="46dp"
            android:text="2000"
            android:textSize="24sp"
            app:layout_constraintBottom_toBottomOf="@+id/motivation"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:id="@+id/listOfLaureates"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:fontFamily="sans-serif-smallcaps"
            android:text="Syukuro Manabe, Klaus Hasselmann, Klaus Hasselmann, Klaus Hasselmann"
            android:textAlignment="textStart"
            android:textSize="18sp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toEndOf="@+id/year"
            app:layout_constraintTop_toTopOf="parent" />

        <TextView
            android:id="@+id/motivation"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:fontFamily="sans-serif-black"
            android:text="for groundbreaking contributions to our understanding of complex physical systems"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toEndOf="@+id/year"
            app:layout_constraintTop_toBottomOf="@+id/listOfLaureates" />

</androidx.constraintlayout.widget.ConstraintLayout>

</androidx.cardview.widget.CardView>

Kolejnym niezbędnym elementem jest adapter dla `RecyclerView` - dodajmy pakiet `adapters.nobelprizes` - skorzystamy z `ListAdapter`, więc musimy utworzyć trzy klasy. Rozpocznijmy od `ViewHolder`

In [ ]:
class NobelPrizeViewHolder(private val binding: NobelPrizeRvItemBinding)
    : RecyclerView.ViewHolder(binding.root){}

Ponieważ dostęp do laureatów nagrody mamy przez listę, a chcemy wyświetlić ich `FullName` w polu `TextView`, dodajmy metodę zwracającą sformatowany `String`

In [ ]:
private fun laureates(item: NobelPrize): String{
    val laureates = StringBuilder()
    item.laureates?
    .forEach { laureates.append(it.fullName?.en?:"no name").append(" ; ") }
        ?: laureates.append("no names")
    return laureates.toString()
}

Tutaj musimy zwrócić uwagę, że pola które otrzymujemy z serwera mogą być puste (`null`), musimy te przypadki również obsłużyć. Dodajmy funkcję `bind`

In [ ]:
fun bind(item: NobelPrize){
    binding.year.text = item.awardYear
    binding.motivation.text = item.topMotivation?.en?:"not specified"
    binding.listOfLaureates.text = laureates(item)
}

Zdefiniujmy `Comparator` oraz `Adapter`

In [ ]:
class NobelPrizeComparator : DiffUtil.ItemCallback<NobelPrize>() {
    override fun areItemsTheSame(oldItem: NobelPrize, newItem: NobelPrize): Boolean {
        return oldItem.awardYear == newItem.awardYear
    }

    override fun areContentsTheSame(oldItem: NobelPrize, newItem: NobelPrize): Boolean {
        return oldItem == newItem
    }
}

In [ ]:
class NobelPrizeAdapter(itemComparator: NobelPrizeComparator) 
    : ListAdapter<NobelPrize, NobelPrizeViewHolder>(itemComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): NobelPrizeViewHolder {
        return NobelPrizeViewHolder(NobelPrizeRvItemBinding.inflate(
            LayoutInflater.from(parent.context), parent, false
        ))
    }

    override fun onBindViewHolder(holder: NobelPrizeViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }
}

Przejdźmy do `NobelAwardsFragment` - dodajmy `viewModel`

In [ ]:
private val nobelPrizesViewModel: NobelPrizesViewModel by viewModels()

Nastrępnie zdefiniujmy funkcję pomocniczą dla `RecyclerView`

In [ ]:
private fun setupRecyclerView(nobelAdapter: NobelPrizeAdapter) {
    binding.nobelPrizeRV.apply {
        adapter = nobelAdapter
        layoutManager = LinearLayoutManager(requireContext())
    }
}

Następnie oddajmy funkcję pomocniczą dla `Spinner`

In [ ]:
private fun setupSpinner() {
    binding.categorySpinner.apply {
        adapter = ArrayAdapter(
            context,
            R.layout.spinner_nobel_award_layout,
            categories.keys.map { it })
        onItemSelectedListener = object : AdapterView.OnItemSelectedListener {
            override fun onItemSelected(
                parent: AdapterView<*>?,
                view: View?,
                position: Int,
                id: Long
            ) {
                nobelPrizesViewModel.getNobelPrizes(categories.values.map { it }[position])
            }

            override fun onNothingSelected(parent: AdapterView<*>?) {}
        }
    }
}

Dodajmy jeszcze dwie funkcje określające widoczność `ProgressBar`

In [ ]:
private fun hideProgressBar(){
    binding.nobelPrizeProgressBar.visibility = View.INVISIBLE
}

private fun showProgressBar(){
    binding.nobelPrizeProgressBar.visibility = View.VISIBLE
}

Finalnie, zdefiniujmy funkcję obsługującą odpowiedź

In [ ]:
private fun observeNoblePrizeList(nobelAdapter: NobelPrizeAdapter) {

W zależności od wyemitowanej odpowiedzi chcemy ustawić fragment w odpowiedni stan

In [ ]:
    nobelPrizesViewModel.noblePrizes.observe(viewLifecycleOwner) { response ->
        when (response) {

Jeżeli naszym stanem jest ładowanie, chcemy pokazać `ProgressBar`

In [ ]:
            is Resource.Loading -> showProgressBar()

Jeżeli dostajemy błąd, chawamy `ProgressBar` i wyświetlamy komunikat - tutaj dla uproszczenia zrobimy to za pomocą `Log`

In [ ]:
            is Resource.Error -> {
                hideProgressBar()
                response.message?.let { Log.e(TAG, "Error occurred: $it") }
            }

Zdefiniujmy stałą `TAG`

In [ ]:
private val TAG = "NobelAwardsFragment"

Jeżeli odpowiedź jest poprawna, chowamy `ProgressBar` i wykonujemy metodę `submitList` na adapterze `RecyclerView`

In [ ]:
            is Resource.Success -> {
                hideProgressBar()
                response.data?.let { nobelAdapter.submitList(it.nobelPrizes) }
            }

W metodzie `onViewCreated` wykonujemy wcześniej zdefiniowane metody

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)

    setupSpinner()

    val nobelAdapter = NobelPrizeAdapter(NobelPrizeComparator())
    setupRecyclerView(nobelAdapter)

    observeNoblePrizeList(nobelAdapter)
}

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/NuuG5HnAdmdTjLV0DV/giphy.webp" width="150" />

### **`NobelAwardFragment`**

Obsłużymy teraz kolejny fragment - wyświetlający pełniejszą informację o zadanej nagrodzie. Musimy jednoznacznie określić nagrodę - ponieważ nie mamy dostępnego identyfikatora, posłużymy się rokiem oraz kategorią.

Z fragmentu `NobelAwardsFragment` prześlemy dwie informacje (`awardYear`, `category`) do `NobelAwardFragment`, następnie połączymy się z serwerem i wykonamy odpowiednie żądanie. Istnieje również inny sposób na osiągnięcie tego samoego rezultatu - ponieważ mamy już dostępne wszystkie dane, możemy również przekazać cały obiekt (tutaj należy wwykorzystać interfejs `Serializable`, lub `Parcelable`).

Po pierwsze dodajmy odpowiednie argumenty do `navigation`.

In [ ]:
<fragment
    android:id="@+id/nobelAwardsFragment"
    android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.fragments.nobleawards.NobelAwardsFragment"
    android:label="Nagrody Nobla"
    tools:layout="@layout/fragment_nobel_awards" >
    <action
        android:id="@+id/action_nobelAwardsFragment_to_nobelAwardFragment"
        app:destination="@id/nobelAwardFragment" >
        <argument
            android:name="category"
            app:argType="string" />
        <argument
            android:name="awardYear"
            app:argType="string" />
    </action>
</fragment>

Dodajmy `onClick` do funkcji `bind` klasy `NobelPrizeViewHolder`

In [ ]:
binding.root.setOnClickListener {
    val action: NavDirections = NobelAwardsFragmentDirections
        .actionNobelAwardsFragmentToNobelAwardFragment(
            category = item.category.en,
            awardYear = item.awardYear
        )
    findNavController(binding.root).navigate(action)
}

Dodajmy funkcję `getNobelPrize` do `NobelPrizeApi`, która przyjmować będzie kategorię i rok

In [ ]:
@GET("2.1/nobelPrizes")
suspend fun getNobelPrize(
    @Query("nobelPrizeYear") year: Int,
    @Query("nobelPrizeCategory") category: String,
) : Response<NobelPrizeResponse>

Dodajmy również odpowiednią metodę do `NobelRepository`

In [ ]:
suspend fun getNobelPrize(year: Int, category: String) = 
    RetrofitInstance.api.getNobelPrize(year = year, category = category)

Przygotujmy layout dla `NobelAwardFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.fragments.nobelaward.NobelAwardFragment">

    <ProgressBar
        android:id="@+id/nobelPrizeProgressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="8dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/dateAwardTextView" />

    <TextView
        android:id="@+id/categoryFullNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginTop="16dp"
        android:fontFamily="sans-serif-smallcaps"
        android:text="Physics"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/dateAwardTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:fontFamily="sans-serif"
        android:text="Date awarded"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/categoryFullNameTextView" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/nobelAwardRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintTop_toBottomOf="@+id/dateAwardTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>

Również tutaj będziemy mieli `RecyclerView` - wyświetlimy w nim listę laureatów nagrody. Dodajmy layout dla pojedynczego elementu.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>

<androidx.cardview.widget.CardView 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp"
    app:cardBackgroundColor="@color/teal_200"
    android:layout_margin="8dp"
    >

<androidx.constraintlayout.widget.ConstraintLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="16dp">


    <TextView
        android:id="@+id/fullNameTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:fontFamily="sans-serif-smallcaps"
        android:text="Full Name"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/textView2"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:text="Portion: "
        android:textSize="20sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/portionTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:text="1/4"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/textView2"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/motivationTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_margin="8dp"
        android:text="motivation"
        android:textSize="16sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/portionTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

Skoro mamy `RecyclerView`, musimy zaimplementować odpowiedni adapter - do pakietu `adapters` dodajmy pakiet `nobelprizelaureates` i tworzymy klasy `LaureateViewHolder`, `LaureateComparator`, `LaureateAdapter`

In [ ]:
class LaureateViewHolder(private val binding: LaureateRvItemBinding)
    : RecyclerView.ViewHolder(binding.root){
        fun bind(item: Laureate){
            binding.fullNameTextView.text = item.fullName?.en?: "missing name"
            binding.portionTextView.text = item.portion
            binding.motivationTextView.text = item.motivation.en
        }
}

In [ ]:
class LaureateComparator : DiffUtil.ItemCallback<Laureate>() {
    override fun areItemsTheSame(oldItem: Laureate, newItem: Laureate): Boolean {
        return oldItem.id == newItem.id
    }

    override fun areContentsTheSame(oldItem: Laureate, newItem: Laureate): Boolean {
        return oldItem == newItem
    }
}

In [ ]:
class LaureateAdapter(itemComparator: LaureateComparator) 
    : ListAdapter<Laureate, LaureateViewHolder>(itemComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): LaureateViewHolder {
        return LaureateViewHolder(
            LaureateRvItemBinding.inflate(
            LayoutInflater.from(parent.context), parent, false
        ))
    }

    override fun onBindViewHolder(holder: LaureateViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }
}

Do pakieru `ui.fragments` dodajmy pakiet `nobelaward` i przenieśmy do niego klasę `NobelAwardFragment`. W tym pakiecie tworzymy również `NobelPrizeViewModel`

In [ ]:
class NobelPrizeViewModel : ViewModel() {
    private val repository = NobelRepository()
    private val _nobelPrize: MutableLiveData<Resource<NobelPrizeResponse>> = MutableLiveData()

    val noblePrize: LiveData<Resource<NobelPrizeResponse>>
        get() = _nobelPrize

    fun getNobelPrize(year: Int, category: String) = viewModelScope.launch {
        _nobelPrize.postValue(Resource.Loading())
        val response = repository.getNobelPrize(year= year, category= category)
        _nobelPrize.postValue(handleNobelPrizesResponse(response))
    }

    private fun handleNobelPrizesResponse(response: Response<NobelPrizeResponse>)
            : Resource<NobelPrizeResponse> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }
}

W `NobelAwardFragment` tworzymy `viewmodel` oraz odbieramy przesłane argumenty.

In [ ]:
class NobelAwardFragment : Fragment() {

    private val nobelPrizeViewModel: NobelPrizeViewModel by viewModels()
    private val TAG = "NobelAwardFragment"

    private val category: String? by lazy { requireArguments().getString("category") }
    private val awardYear: String? by lazy { requireArguments().getString("awardYear") }
    ...
}

Podobnie jak w poprzednim fragmencie dodajmy kilka funkcji pomocniczych

In [ ]:
private fun hideProgressBar(){
    binding.nobelPrizeProgressBar.visibility = View.INVISIBLE
}

private fun showProgressBar(){
    binding.nobelPrizeProgressBar.visibility = View.VISIBLE
}

private fun setupRecyclerView(laureateAdapter: LaureateAdapter) {
    binding.nobelAwardRV.apply {
        adapter = laureateAdapter
        layoutManager = LinearLayoutManager(requireContext())
    }
}

private fun observeNoblePrize(laureateAdapter: LaureateAdapter) {
    nobelPrizeViewModel.noblePrize.observe(viewLifecycleOwner) { response ->
        when (response) {
            is Resource.Success -> {
                hideProgressBar()
                response.data?.let {
                    binding.categoryFullNameTextView.text = 
                        it.nobelPrizes.first().categoryFullName.en
                    binding.dateAwardTextView.text = it.nobelPrizes.first().dateAwarded
                    laureateAdapter.submitList(it.nobelPrizes.first().laureates)
                }
            }
            is Resource.Error -> {
                hideProgressBar()
                response.message?.let { Log.e(TAG, "Error occurred: $it") }
            }
            is Resource.Loading -> showProgressBar()
        }
    }
}

W przypadku poprawnej odpowiedzi ustawiamy odpowiednie pola, oraz przesyłamy listę laureatów do adaptera `RecyclerView`.

Zwróćmy uwagę na to że `nobelPrizes` jest listą

In [ ]:
data class NobelPrizeResponse(
    val meta: Meta,
    val nobelPrizes: List<NobelPrize>
)

Ale wiemy że w tym przypadku dostaniemy zawsze jeden element w liście, więc posługujemy się funkcją `first` aby uzyskać do niego dostęp

Możemy przetestować aplikację

<img src="https://media3.giphy.com/media/6pIR2WozCb81CcuGZo/giphy.webp" width="150" />

### **`LaureateFragment`**

Fragment będzie wyświetlał informacje o wybranym laureacie. W `NobelAwardFragment` mamy listę laureatów jednej, konkretnej nagrody nobla. Dodamy obsługę `onClick` i przekażemy `id` laureata do `LaureateFragment`, który wyświetli pełniejsze informacje. Tym razem skorzystamy z innego endpointa **/laureate/{laureateID}**. Przykładowe żądanie: http://api.nobelprize.org/2.1/laureate/11

Tutaj mamy tylko jeden parametr
```xml
- name: `laureateID`  
  in: path  
  required: `true`  
  description: ID laureata.  
  type: `integer`  
  minimum: 1
```

Odpowiedź wygląda następująco

In [ ]:
[
  {
    "id": "11",
    "knownName": {
      "en": "Albert A. Michelson",
      "se": "Albert A. Michelson"
    },
    "givenName": {
      "en": "Albert A.",
      "se": "Albert A."
    },
    "familyName": {
      "en": "Michelson",
      "se": "Michelson"
    },
    "fullName": {
      "en": "Albert Abraham Michelson",
      "se": "Albert Abraham Michelson"
    },
    "fileName": "michelson",
    "gender": "male",
    "birth": {
      "date": "1852-12-19",
      "place": {
        "city": {
          "en": "Strelno",
          "no": "Strelno",
          "se": "Strelno"
        },
        "country": {
          "en": "Prussia",
          "no": "Preussen",
          "se": "Preussen"
        },
        "cityNow": {
          "en": "Strzelno",
          "no": "Strzelno",
          "se": "Strzelno",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q1005414",
            "https://www.wikipedia.org/wiki/Strzelno"
          ]
        },
        "countryNow": {
          "en": "Poland",
          "no": "Polen",
          "se": "Polen",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q36"
          ]
        },
        "continent": {
          "en": "Europe",
          "no": "Europa",
          "se": "Europa"
        },
        "locationString": {
          "en": "Strelno, Prussia (now Strzelno, Poland)",
          "no": "Strelno, Preussen (nå Strzelno, Polen)",
          "se": "Strelno, Preussen (nu Strzelno, Polen)"
        }
      }
    },
    "death": {
      "date": "1931-05-09",
      "place": {
        "city": {
          "en": "Pasadena, CA",
          "no": "Pasadena, CA",
          "se": "Pasadena, CA"
        },
        "country": {
          "en": "USA",
          "no": "USA",
          "se": "USA",
          "sameAs": "https://www.wikidata.org/wiki/Q30"
        },
        "cityNow": {
          "en": "Pasadena, CA",
          "no": "Pasadena, CA",
          "se": "Pasadena, CA",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q485176",
            "https://www.wikipedia.org/wiki/Pasadena,_California"
          ]
        },
        "countryNow": {
          "en": "USA",
          "no": "USA",
          "se": "USA",
          "sameAs": [
            "https://www.wikidata.org/wiki/Q30"
          ]
        },
        "continent": {
          "en": "North America",
          "no": "Nord-Amerika",
          "se": "Nordamerika"
        },
        "locationString": {
          "en": "Pasadena, CA, USA",
          "no": "Pasadena, CA, USA",
          "se": "Pasadena, CA, USA"
        }
      }
    },
    "wikipedia": {
      "slug": "Albert_Abraham_Michelson",
      "english": "https://en.wikipedia.org/wiki/Albert_Abraham_Michelson"
    },
    "wikidata": {
      "id": "Q127234",
      "url": "https://www.wikidata.org/wiki/Q127234"
    },
    "sameAs": [
      "https://www.wikidata.org/wiki/Q127234",
      "https://en.wikipedia.org/wiki/Albert_Abraham_Michelson"
    ],
    "links": [
      {
        "rel": "laureate",
        "href": "https://api.nobelprize.org/2/laureate/11",
        "action": "GET",
        "types": "application/json"
      },
      {
        "rel": "external",
        "href": "https://www.nobelprize.org/laureate/11",
        "title": "Albert A. Michelson - Facts",
        "action": "GET",
        "types": "text/html",
        "class": [
          "laureate facts"
        ]
      }
    ],
    "nobelPrizes": [
      {
        "awardYear": "1907",
        "category": {
          "en": "Physics",
          "no": "Fysikk",
          "se": "Fysik"
        },
        "categoryFullName": {
          "en": "The Nobel Prize in Physics",
          "no": "Nobelprisen i fysikk",
          "se": "Nobelpriset i fysik"
        },
        "sortOrder": "1",
        "portion": "1",
        "prizeStatus": "received",
        "motivation": {
          "en": "for his optical precision instruments and the spectroscopic and metrological investigations carried out with their aid",
          "se": "för hans optiska precisionsinstrument och hans därmed utförda spektroskopiska och metrologiska undersökningar"
        },
        "prizeAmount": 138796,
        "prizeAmountAdjusted": 7161123,
        "affiliations": [
          {
            "name": {
              "en": "University of Chicago",
              "no": "University of Chicago",
              "se": "University of Chicago"
            },
            "nameNow": {
              "en": "University of Chicago"
            },
            "city": {
              "en": "Chicago, IL",
              "no": "Chicago, IL",
              "se": "Chicago, IL"
            },
            "country": {
              "en": "USA",
              "no": "USA",
              "se": "USA"
            },
            "cityNow": {
              "en": "Chicago, IL",
              "no": "Chicago, IL",
              "se": "Chicago, IL",
              "sameAs": [
                "https://www.wikidata.org/wiki/Q1297",
                "https://www.wikipedia.org/wiki/Chicago"
              ]
            },
            "countryNow": {
              "en": "USA",
              "no": "USA",
              "se": "USA",
              "sameAs": [
                "https://www.wikidata.org/wiki/Q30"
              ]
            },
            "locationString": {
              "en": "Chicago, IL, USA",
              "no": "Chicago, IL, USA",
              "se": "Chicago, IL, USA"
            }
          }
        ],
        "links": [
          {
            "rel": "nobelPrize",
            "href": "https://api.nobelprize.org/2/nobelPrize/phy/1907",
            "action": "GET",
            "types": "application/json"
          },
          {
            "rel": "external",
            "href": "https://www.nobelprize.org/prizes/physics/1907/michelson/facts/",
            "title": "Albert A. Michelson - Facts",
            "action": "GET",
            "types": "text/html",
            "class": [
              "laureate facts"
            ]
          },
          {
            "rel": "external",
            "href": "https://www.nobelprize.org/prizes/physics/1907/summary/",
            "title": "The Nobel Prize in Physics 1907",
            "action": "GET",
            "types": "text/html",
            "class": [
              "prize summary"
            ]
          }
        ]
      }
    ],
    "meta": {
      "terms": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/",
      "license": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#licence",
      "disclaimer": "https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#disclaimer"
    }
  }
]

Również tutaj skorzystamy z pluginy `JsonToKotlinClass`, który automatycznie generuje nam klasy

In [ ]:
class LaureateResponse : ArrayList<LaureateResponseItem>()

data class LaureateResponseItem(
    val birth: Birth,
    val death: Death?,
    val familyName: FamilyName,
    val fileName: String,
    val fullName: FullName,
    val gender: String,
    val givenName: GivenName,
    val id: String,
    val knownName: KnownName,
    val links: List<Link>,
    val meta: Meta,
    val nobelPrizes: List<NobelPrize>,
    val sameAs: List<String>,
    val wikidata: Wikidata,
    val wikipedia: Wikipedia
)

data class Birth(
    val date: String,
    val place: Place
)

data class Death(
    val date: String?,
    val place: PlaceX
)

data class FamilyName(
    val en: String,
    val se: String
)

data class FullName(
    val en: String,
    val se: String
)

data class GivenName(
    val en: String,
    val se: String
)

data class KnownName(
    val en: String,
    val se: String
)

data class Link(
    val action: String,
    val `class`: List<String>,
    val href: String,
    val rel: String,
    val title: String,
    val types: String
)

data class Place(
    val city: City,
    val cityNow: CityNow,
    val continent: Continent,
    val country: Country,
    val countryNow: CountryNow,
    val locationString: LocationString
)

data class PlaceX(
    val city: City,
    val cityNow: CityNow,
    val continent: Continent,
    val country: CountryX,
    val countryNow: CountryNow,
    val locationString: LocationString
)

data class Meta(
    val disclaimer: String,
    val license: String,
    val terms: String
)

data class Category(
    val en: String,
    val no: String,
    val se: String
)

data class CategoryFullName(
    val en: String,
    val no: String,
    val se: String
)

data class Country(
    val en: String,
    val no: String,
    val se: String
)

data class CountryNow(
    val en: String,
    val no: String,
    val sameAs: List<String>,
    val se: String
)

data class LocationString(
    val en: String,
    val no: String,
    val se: String
)

data class Motivation(
    val en: String,
    val se: String
)

data class Name(
    val en: String,
    val no: String,
    val se: String
)

data class NameNow(
    val en: String
)

data class NobelPrize(
    val affiliations: List<Affiliation>?,
    val awardYear: String,
    val category: Category,
    val categoryFullName: CategoryFullName,
    val links: List<Link>,
    val motivation: Motivation,
    val portion: String,
    val prizeAmount: Int,
    val prizeAmountAdjusted: Int,
    val prizeStatus: String,
    val sortOrder: String
)

data class Affiliation(
    val city: City,
    val cityNow: CityNow,
    val country: Country,
    val countryNow: CountryNow,
    val locationString: LocationString,
    val name: Name,
    val nameNow: NameNow
)

data class City(
    val en: String,
    val no: String,
    val se: String
)

data class CityNow(
    val en: String,
    val no: String,
    val sameAs: List<String>,
    val se: String
)

data class Continent(
    val en: String,
    val no: String,
    val se: String
)

data class CountryX(
    val en: String,
    val no: String,
    val sameAs: String,
    val se: String
)

data class Wikidata(
    val id: String,
    val url: String
)

data class Wikipedia(
    val english: String,
    val slug: String
)

Jak widać dostaliśmy koło 40 klas - jak wspominałem na początku nie będziemy specjalnie się przejmować tym co się dzieje dokładnie w tych klasach, więc zostawmy je tak jak zostały wygenerowane.

Zwróćmy uwagę na dwie pierwsze klasy
- `LaureateResponse : ArrayList<LaureateResponseItem>()` - odpowiedź dostajemy jako listę
- `LaureateResponseItem` - element listy

Dodajmy argument do `navigation`

In [ ]:
<fragment
    android:id="@+id/nobelAwardFragment"
    android:name="pl.udu.uwr.pum.verynobleappkotlin.ui.fragments.nobelaward.NobelAwardFragment"
    android:label="Nagroda Nobla"
    tools:layout="@layout/fragment_nobel_award" >
    <action
        android:id="@+id/action_nobelAwardFragment_to_nobelAwardsFragment"
        app:destination="@id/nobelAwardsFragment" />
    <action
        android:id="@+id/action_nobelAwardFragment_to_laureateFragment"
        app:destination="@id/laureateFragment" >
        <argument
            android:name="id"
            app:argType="string" />
    </action>
</fragment>

Dodajmy również `onClick` do funkcji `bind` klasy `LaureateViewHolder`

In [ ]:
binding.root.setOnClickListener {
    val action: NavDirections = NobelAwardFragmentDirections
        .actionNobelAwardFragmentToLaureateFragment(item.id)
    Navigation.findNavController(binding.root).navigate(action)
}

Przygotujmy layout `LaureateFragment`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:layout_margin="8dp">

    <ProgressBar
        android:id="@+id/laureateProgressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="8dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/fullNameTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:fontFamily="sans-serif-smallcaps"
        android:gravity="center"
        android:text="Full Name"
        android:textSize="24sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/birth"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Birth:"
        android:textSize="20sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/birthDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:text="19.01.2000"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/birth"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/death"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="8dp"
        android:text="Death:"
        android:textSize="20sp"
        app:layout_constraintEnd_toStartOf="@+id/deathDateTextView"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/deathDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="19.01.2000"
        android:textSize="20sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/fullNameTextView" />

    <TextView
        android:id="@+id/birthCityTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:text="Warszawa"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/birth"
        app:layout_constraintTop_toBottomOf="@+id/birthDateTextView" />

    <TextView
        android:id="@+id/deathCityTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="8dp"
        android:text="Warszawa"
        android:textSize="20sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/deathDateTextView" />

    <TextView
        android:id="@+id/birthCountryTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:text="Poland"
        android:textSize="20sp"
        app:layout_constraintStart_toEndOf="@+id/birth"
        app:layout_constraintTop_toBottomOf="@+id/birthCityTextView" />

    <TextView
        android:id="@+id/deathCountryTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="Poland"
        android:textSize="20sp"
        android:layout_marginEnd="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/deathCityTextView" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/laureateRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/birthCountryTextView" />

    <Button
        android:id="@+id/wikiButton"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="wikipedia"
        android:layout_marginStart="36dp"
        android:layout_marginEnd="36dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/laureateRV" />

</androidx.constraintlayout.widget.ConstraintLayout>

Laureat może posiadać kilka nagród nobla, więc dodajmy również tutaj `RecyclerView` z listą wszystkich nagród otrzymanych przez laureata. W dostępnych danych mamy również link do strony na wikipedi, wykorzystamy to na kolejnym fragmencie, do którego przejdziemy dzięki dodanemu przyciskowi.

Dodajmy również layout pojedynczego elementu listy

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.cardview.widget.CardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    app:cardCornerRadius="30dp"
    app:cardElevation="15dp"
    app:cardBackgroundColor="@color/teal_200"
    android:layout_margin="8dp"
    >

<androidx.constraintlayout.widget.ConstraintLayout
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="16dp">

    <TextView
        android:id="@+id/yearTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="2000"
        android:textSize="20sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/categoryFullNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:layout_marginBottom="3dp"
        android:text="Physics"
        android:textSize="20sp"
        app:layout_constraintBottom_toBottomOf="@+id/yearTextView"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toEndOf="@+id/yearTextView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/motivationTextView"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:layout_marginStart="8dp"
        android:lines="4"
        android:text="for groundbreaking contributions to our understanding of complex physical systems"
        android:textSize="12sp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toEndOf="@+id/portion"
        app:layout_constraintTop_toBottomOf="@+id/affiliationsTextView" />

    <TextView
        android:id="@+id/portion"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="4dp"
        android:text="Portion:"
        android:textSize="8sp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/affiliationsTextView" />

    <TextView
        android:id="@+id/portionTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:text="1/2"
        android:textSize="20sp"
        android:layout_marginStart="4dp"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/portion" />

    <TextView
        android:id="@+id/affiliationsTextView"
        android:layout_width="0dp"
        android:layout_height="wrap_content"
        android:gravity="center"
        android:text="Uniwersytet Wroclawski"
        android:textSize="20sp"
        app:layout_constraintEnd_toEndOf="parent"

        app:layout_constraintStart_toEndOf="@+id/affiliations"
        app:layout_constraintTop_toBottomOf="@+id/categoryFullNameTextView" />

    <TextView
        android:id="@+id/affiliations"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginBottom="4dp"
        android:text="Affiliations:"
        android:textSize="16sp"
        app:layout_constraintBottom_toTopOf="@+id/motivationTextView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toBottomOf="@+id/yearTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
</androidx.cardview.widget.CardView>

Oraz przygotujmy adapter. Dodajmy pakiet `laureatenobelprize` do pakietu `adapters` i zaimplementujmy odpowiednie klasy

In [ ]:
class NobelPrizeViewHolder(private val binding: LaureateNobelPrizeRvItemBinding)
    : RecyclerView.ViewHolder(binding.root) {
    fun bind(item: NobelPrize){
        binding.yearTextView.text = item.awardYear
        binding.portionTextView.text = item.portion
        binding.motivationTextView.text = item.motivation.en
        binding.categoryFullNameTextView.text = item.categoryFullName.en
        binding.affiliationsTextView.text = affiliations(item)
    }

    private fun affiliations(item: NobelPrize): String{
        val affiliations = StringBuilder()
        item.affiliations?.forEach { affiliations.append(it.name.en).append("\n") }
        return affiliations.toString()
    }
}

Wyświetlamy nazwy instytutów z którymi jest powiązana dana osoba. Podobnie jak w przypadku `NobelPrizesViewHolder` dodajmy dodatkową metodę - mamy dostępną listę wszystkich instytutów, chcemy `String`.

In [ ]:
class NobelPrizeComparator : DiffUtil.ItemCallback<NobelPrize>() {
    override fun areItemsTheSame(oldItem: NobelPrize, newItem: NobelPrize): Boolean {
        return oldItem === newItem
    }

    override fun areContentsTheSame(oldItem: NobelPrize, newItem: NobelPrize): Boolean {
        return oldItem == newItem
    }
}

In [ ]:
class NobelPrizeAdapter(itemComparator: NobelPrizeComparator) 
    : ListAdapter<NobelPrize, NobelPrizeViewHolder>(itemComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int)
        : NobelPrizeViewHolder {
        return NobelPrizeViewHolder(
            LaureateNobelPrizeRvItemBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )
    }

    override fun onBindViewHolder(holder: NobelPrizeViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }
}

Do interfejsu `NobelPrizeApi` dodajmy nową funkcję

In [ ]:
@GET("2.1/laureate/{laureateID}")
suspend fun getLaureates(@Path("laureateID") id: String) 
    : Response<LaureateResponse>

dodajmy również `getLaureate` do repozytorium

In [ ]:
suspend fun getLaureate(id: String) = RetrofitInstance.api.getLaureates(id)

Do pakietu `ui.fragmenst.laureate` dodajmy `LaureateViewModel` - jest on analogiczny do poprzednich

In [ ]:
class LaureateViewModel : ViewModel() {
    private val repository = NobelRepository()
    private val _laureate: MutableLiveData<Resource<LaureateResponse>> = MutableLiveData()

    val laureate: LiveData<Resource<LaureateResponse>>
        get() = _laureate

    fun getLaureate(id: String) = viewModelScope.launch {
        _laureate.postValue(Resource.Loading())
        val response = repository.getLaureate(id)
        _laureate.postValue(handleLaureateResponse(response))
    }

    private fun handleLaureateResponse(response: Response<LaureateResponse>)
            : Resource<LaureateResponse> {
        if (response.isSuccessful)
            response.body()?.let { return Resource.Success(it) }
        return Resource.Error(response.message())
    }
}

W `LaureateFragment` odbieramy przesłany argument

In [ ]:
private val id: String? by lazy { requireArguments().getString("id") }

Podobnie jak w poprzednich fragmentach, dodajmy kilka funkcji

In [ ]:
private fun hideProgressBar(){
    binding.laureateProgressBar.visibility = View.INVISIBLE
}

private fun showProgressBar(){
    binding.laureateProgressBar.visibility = View.VISIBLE
}

private fun observeLaureatePrize(nobelPrizeAdapter: NobelPrizeAdapter) {
    laureateViewModel.laureate.observe(viewLifecycleOwner) { response ->
        when (response) {
            is Resource.Success -> {
                hideProgressBar()
                response.data?.let { res ->
                    inflateLayoutWithDate(res[0]) // res jest typu ArrayList
                    nobelPrizeAdapter.submitList(res[0].nobelPrizes)

                    binding.wikiButton.setOnClickListener {
                        val action: NavDirections = LaureateFragmentDirections
                            .actionLaureateFragmentToWikiLaureateFragment(
                                res[0].wikipedia.english
                            )
                        findNavController().navigate(action)
                    }
                }
            }
            is Resource.Error -> {
                hideProgressBar()
                response.message?.let { Log.e(TAG, "Error occurred: $it") }
            }
            is Resource.Loading -> showProgressBar()
        }
    }
}

private fun inflateLayoutWithDate(item: LaureateResponseItem){
    binding.fullNameTextView.text = item.fullName.en
    binding.birthDateTextView.text = item.birth.date
    binding.birthCityTextView.text = item.birth.place.city.en
    binding.birthCountryTextView.text = item.birth.place.country.en
    binding.deathDateTextView.text = item.death?.date?:""
    binding.deathCityTextView.text = item.death?.place?.city?.en?:""
    binding.deathCountryTextView.text = item.death?.place?.country?.en?:""
}

private fun setupRecyclerView(nobelPrizeAdapter: NobelPrizeAdapter) {
    binding.laureateRV.apply {
        adapter = nobelPrizeAdapter
        layoutManager = LinearLayoutManager(requireContext())
    }
}

Nasze dane otrzymujemy jako `ArrayList`, więc używamy odpowiednich metod dostępowych. Dodajemy również obsługę `onClick` naszego przycisku, dzięki któremu przechodzimy na `WikiFragment` - przekazujemy również `url` do wpisu w wikipedii.

### **`WikiFragment`**

W tym fragmencie wyświetlimy stronę internetową wiki - zrobimy to wykorzystując `WebView`. Rozpocznijmy od layoutu.

In [ ]:
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".ui.fragments.wiki.WikiLaureateFragment">

    <WebView
        android:id="@+id/webView"
        android:layout_margin="8dp"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>

</FrameLayout>

W `WikiFragment` odbieramy argument

In [ ]:
private val wikiUrl: String? by lazy { requireArguments().getString("url") }

I w metodzie `onViewCreated` inicjujemy `WebView`

In [ ]:
binding.webView.apply {
    webViewClient = WebViewClient()
    loadUrl(wikiUrl!!)
}

W pierwszym kroku tworzymy `WebViewClient` - jest to klasa która pozwala aplikacji przejąć kontrolę przy ładowaniu `url` do `WebView`. Jeżeli klient nie jest zainicjowany, `WebView` przekazuje kontrolę do `ActivityManager`, aby ten wybrał odpowiedni handler dla tego typu zasobu.

Następnie wykorzystujemy `loeadUrl` do załadowania adresu i wyświetlenia treści.

Możemy przetestować aplikację

<table><tr><td><img src="https://media0.giphy.com/media/NuuG5HnAdmdTjLV0DV/giphy.webp" width="150" /></td><td><img src="https://media3.giphy.com/media/6pIR2WozCb81CcuGZo/giphy.webp" width="150" /></td><td><img src="https://media0.giphy.com/media/8QoE7dYN8as4gRjE16/giphy.webp" width="150" /></td></tr></table>